In [2]:
import h2o
h2o.init()

Checking whether there is an H2O instance running at http://localhost:54321. connected.


H2O cluster uptime:,15 secs
H2O cluster version:,3.16.0.4
H2O cluster version age:,1 month and 2 days
H2O cluster name:,root
H2O cluster total nodes:,1
H2O cluster free memory:,3.544 Gb
H2O cluster total cores:,2
H2O cluster allowed cores:,2
H2O cluster status:,"locked, healthy"
H2O connection url:,http://localhost:54321
H2O connection proxy:,None
